In [2]:
import os
import re
import sys
import pandas as pd
from tqdm.auto import tqdm

In [3]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
from src.utils import load_dataset
from src.search_tf_idf.search import TfidfSearch
from src.classic_keywords.preprocessing import preprocess_text
from src.classic_keywords.preprocessing import lemmatize_word 


df = load_dataset("df_with_llmkeyphrases")

Данные успешно загружены из ../data/df_with_llmkeyphrases.parquet. Количество записей: 200


In [ ]:
pd.set_option('display.max_colwidth', 300)
tqdm.pandas()

In [10]:
tfidf_search_engine = TfidfSearch()

tfidf_search_engine.build_index(
    lemmatized_texts=df['lemmatized_abstract'],
    original_texts=df['abstract']
)

Building TF-IDF index...
TF-IDF index built successfully.


In [11]:
def highlight_keywords(text: str, query: str, preprocessor_func) -> str:
    """
    Выделяет в оригинальном тексте слова, леммы которых присутствуют в запросе.
    """
    query_lemmas = set(preprocessor_func(query).split())
    
    tokens = re.split(r'(\s+)', text) 
    
    highlighted_tokens = []
    for token in tokens:
        if not token.strip():
            highlighted_tokens.append(token)
            continue
        
        cleaned_token = token.strip('.,!?:;()[]{}')
        
        if lemmatize_word(cleaned_token.lower()) in query_lemmas:
            highlighted_tokens.append(f"**{token}**")
        else:
            highlighted_tokens.append(token)
            
    return "".join(highlighted_tokens)

from IPython.display import display, Markdown

Для сравнения методов мы составим поисковые запросы связанные с документом индекса 94. 

* Запрос: высокотехнологичная медицинская помощь сердечно-сосудистая хирургия ДВФО
* Запрос: кардиохирургия ВМП Дальний Восток
* Запрос: сравнение организации высокотехнологичных операций на сердце и сосудах в регионах Дальнего Востока

In [13]:
queries = [
    "высокотехнологичная медицинская помощь сердечно-сосудистая хирургия ДВФО",
    "кардиохирургия ВМП Дальний Восток",
    "сравнение организации высокотехнологичных операций на сердце и сосудах в регионах Дальнего Востока"
]

for query in queries:
    print(f"--- Результаты поиска для запроса: '{query}' ---")
    
    search_results = tfidf_search_engine.search(query, preprocessor_func=preprocess_text)
    
    if not search_results:
        print("Ничего не найдено.")
    else:
        for i, (doc_index, doc_text) in enumerate(search_results):
            highlighted = highlight_keywords(doc_text, query, preprocess_text)
            md_output = f"""{i+1}. (Индекс документа: {doc_index})
            > {highlighted[:500]}..."""
            display(Markdown(md_output))

--- Результаты поиска для запроса: 'высокотехнологичная медицинская помощь сердечно-сосудистая хирургия ДВФО' ---


1. (Индекс документа: 94)
            > Проведен сравнительный анализ организации и оказания **высокотехнологичной** **медицинской** **помощи** больным с заболеваниями сердца и сосудов в различных субъектах Дальневосточного федерального округа **(ДВФО)** РФ, а также анализ работы Отдела **сердечно-сосудистой** **хирургии** РБ № 1 — НЦМ по осуществлению **высокотехнологичной** **медицинской** **помощи** по профилю «сердечно-сосудистая хирургия». Обсуждены результаты работы в период с 2007 по 2009 гг. в клиниках **ДВФО** РФ: объемы хиру...

2. (Индекс документа: 180)
            > В данном исследовании доказана эффективность применения в отдаленном периоде двух новых малоинвазивных методик для возможности оперативного выявления, адекватного и интенсивного лечения острого перикардита. Методики, выполняются с **помощью** перикардиоскопии: установка в полость перикарда микро дренажа; выполнение перикардиоскопии без больших разрезов кожи с последующим дренированием с **помощью** перикардиоскопа...

3. (Индекс документа: 112)
            > Предложены варианты построения иерархической структуры диагнозов, используемых комбинированным решающим правилом в процессе постановки уточняемого диагноза в компьютерных системах **медицинской** диагностики. Выявлены их преимущества и недостатки. Описан способ хранения сформированной структуры в виде знаний системы поддержки принятия решений. Ил.: 4. Библиогр.: 10 назв....

4. (Индекс документа: 188)
            > В статье приведены данные о лечении больных с повреждениями передней и боковой стенок верхнечелюстного синуса при различных травматических переломах скулоглазничного комплекса в отделении челюстно-лицевой **хирургии** Первого МГМУ им. И.М. Сеченова. Описаны методики применения имплантатов из тканей никелида титана и перспективы его использования при данной патологии, в частности для передней и боковой стенок верхнечелюстного синуса. Описан клинический случай применения данного материала....

...

1. (Индекс документа: 94)
            > Проведен сравнительный анализ **организации** и оказания **высокотехнологичной** медицинской помощи больным с заболеваниями **сердца** и **сосудов** в различных субъектах Дальневосточного федерального округа (ДВФО) РФ, а также анализ работы Отдела сердечно-сосудистой хирургии РБ № 1 — НЦМ по осуществлению **высокотехнологичной** медицинской помощи по профилю «сердечно-сосудистая хирургия». Обсуждены результаты работы в период с 2007 по 2009 гг. в клиниках ДВФО РФ: объемы хирургической деятельнос...

2. (Индекс документа: 177)
            > В статье нашли отражение результаты проведенного автором исследования процесса решения социальных проблем в Дальневосточном **регионе** в 1918-1922 гг. На конкретных примерах рассмотрена специфика становления системы социального обеспечения населения, в том числе и военнослужащих, на **Дальнем** **Востоке.** Автор, на основе неопубликованных материалов, показывает, как государственные органы использовали социальную политику в качестве инструмента в политической борьбе. Рассмотренные в статье мат...

3. (Индекс документа: 0)
            > Представлен усовершенствованный метод парных **сравнений,** в котором посредством табличных форм систематизированы правила логических выводов при **сравнении** технических систем и формулы проверочных значений. Для этого сформулированы рациональные правила логических выводов при парном **сравнении** систем. С целью проверки результатов оценки на непротиворечивость введены понятия количества баллов, набранных одной системой, и коэффициента качества систем, а также разработаны формулы расчетов. Дл...

4. (Индекс документа: 65)
            > Одним из самых выдающихся научных достижений в области тромболизиса является разработка, создание и внедрение фибринолизина - первого советского препарата, разрушающего тромбы. Внутрикоронарное введение фибринолизина позволило снизить летальность больных инфарктом миокарда почти на 20%. За исследования в этой области в 1982 г. Евгений Иванович Чазов был удостоен Ленинской премии. В течение последующих десятилетий под его руководством в Кардиоцентре созданы научные и клинические лаборатории, в ко...

5. (Индекс документа: 156)
            > Актуальность. Развитие облитерирующего тромбангиита (ОТА) в молодом возрасте, высокий риск инвалидизации пациентов делают проблему лечения и профилактики ОТА актуальной и социально значимой.Цель работы. Выявить факторы, определяющие прогноз ОТА.Методы и материалы. Под нашим наблюдением в ревматологическом отделении ОКБ с 2013 по 2015гг. находилось 49 больных со средним возрастом начала заболевания 39 лет (соотношение мужчин и женщин составило 12:1). Все пациенты были курильщиками. Основной патол...

* Как видно из подсветки, TF-IDF успешно находит документы, содержащие точные ключевые слова из запроса, такие как 'модернизация' и 'производственных'. Ранжирование происходит на основе частоты и редкости этих слов.
* Однако этот подход полностью игнорирует семантику. Если в запросе указано 'модернизация производства', а в тексте — 'оптимизация технологических процессов на заводе', TF-IDF, скорее всего, не найдет этот документ или даст ему низкий ранг.

Выделим 15 абстрактов

In [ ]:
df.head(15)[["abstract"]].to_csv("../data/first_15_abstracts.csv", index=False)